In [50]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pycountry

In [18]:
link_projeto = ("https://raw.githubusercontent.com/guilhermeonrails/data-jobs/refs/heads/main/salaries.csv")

In [24]:
colunas_traduzidas = {
    'work_year': 'ano',
    'experience_level': 'senioridade',
    'employment_type': 'contrato',
    'job_title': 'cargo',
    'salary': 'salario',
    'salary_currency': 'moeda',
    'salary_in_usd': 'usd',
    'employee_residence': 'residencia',
    'remote_ratio': 'remoto',
    'company_location': 'empresa',
    'company_size': 'porte_empresa'
}

senioridade = {
'SE': "Senior",
'MI': "Pleno",
'EN': "Junior",
'EX': "Executivo"
}

contrato = {
'FT': "Integral",
'PT': "Parcial",
'FL': "Freelancer",
'CT': "Contrato"
}

remoto = {
0: "Presencial",
100: "Remto",
50: "Hibrido"
}

empresa = {
'S': "Pequena",
'M': "Media",
'L': "Grande"
}



df = pd.read_csv(link_projeto)



df.rename(columns= colunas_traduzidas, inplace=True)


df['senioridade']= df['senioridade'].replace(senioridade)
df['contrato']= df['contrato'].replace(contrato)
df['remoto']= df['remoto'].replace(remoto)
df['porte_empresa']= df['porte_empresa'].replace(empresa)


dflimpo = df.dropna()
dflimpo = dflimpo.assign(ano = dflimpo['ano'].astype('int64'))

In [30]:
dflimpo.columns

Index(['ano', 'senioridade', 'contrato', 'cargo', 'salario', 'moeda', 'usd',
       'residencia', 'remoto', 'empresa', 'porte_empresa'],
      dtype='object')

In [32]:
dflimpo

,ano,senioridade,contrato,cargo,salario,moeda,usd,residencia,remoto,empresa,porte_empresa
0,2025,Senior,Integral,Data Scientist,214000,USD,214000,US,Remto,US,Media
1,2025,Senior,Integral,Data Scientist,136000,USD,136000,US,Remto,US,Media
2,2025,Pleno,Integral,Data Scientist,158800,USD,158800,AU,Presencial,AU,Media
3,2025,Pleno,Integral,Data Scientist,139200,USD,139200,AU,Presencial,AU,Media
4,2025,Junior,Integral,Data Scientist,90000,USD,90000,US,Presencial,US,Media
...,...,...,...,...,...,...,...,...,...,...,...
133344,2020,Senior,Integral,Data Scientist,412000,USD,412000,US,Remto,US,Grande
133345,2021,Pleno,Integral,Data Scientist,151000,USD,151000,US,Remto,US,Grande
133346,2020,Junior,Integral,Data Scientist,105000,USD,105000,US,Remto,US,Pequena
133347,2020,Junior,Contrato,Data Scientist,100000,USD,100000,US,Remto,US,Grande


In [40]:
dfcientista = dflimpo[dflimpo['cargo'] == 'Data Scientist']
dfmedia = dfcientista.groupby('residencia')['usd'].mean().reset_index()
dfmedia

,residencia,usd
0,AD,50745.000000
1,AE,88333.333333
2,AM,45453.200000
3,AR,78856.641026
4,AS,45555.000000
...,...,...
97,VE,192500.000000
98,VN,57200.000000
99,XK,41052.000000
100,ZA,65204.641791


In [52]:
paises = dfmedia['residencia'].unique().tolist()
paises

['AD',
 'AE',
 'AM',
 'AR',
 'AS',
 'AT',
 'AU',
 'BA',
 'BE',
 'BG',
 'BM',
 'BO',
 'BR',
 'CA',
 'CD',
 'CF',
 'CH',
 'CL',
 'CN',
 'CO',
 'CR',
 'CY',
 'CZ',
 'DE',
 'DK',
 'DO',
 'DZ',
 'EC',
 'EE',
 'EG',
 'ES',
 'FI',
 'FR',
 'GB',
 'GE',
 'GH',
 'GR',
 'HK',
 'HN',
 'HR',
 'HU',
 'ID',
 'IE',
 'IL',
 'IN',
 'IQ',
 'IR',
 'IT',
 'JE',
 'JM',
 'JO',
 'JP',
 'KE',
 'KR',
 'KW',
 'LB',
 'LS',
 'LT',
 'LU',
 'LV',
 'MD',
 'MK',
 'MT',
 'MU',
 'MX',
 'MY',
 'NG',
 'NL',
 'NO',
 'NZ',
 'OM',
 'PA',
 'PE',
 'PH',
 'PK',
 'PL',
 'PR',
 'PT',
 'QA',
 'RO',
 'RS',
 'RU',
 'RW',
 'SA',
 'SE',
 'SG',
 'SI',
 'SK',
 'SV',
 'TH',
 'TN',
 'TR',
 'TW',
 'UA',
 'UG',
 'US',
 'UZ',
 'VE',
 'VN',
 'XK',
 'ZA',
 'ZM']

In [60]:
import pycountry

# Sua lista de países (use a lista real que você obteve)
paises = dfmedia['residencia'].unique().tolist()
paises

# Dicionário para armazenar a conversão
iso3_dict = {}

for iso2_code in paises:
    try:
        # Busca o país pelo código ISO-A2
        pais = pycountry.countries.get(alpha_2=iso2_code)
        if pais:
            # Adiciona a conversão ao dicionário
            iso3_dict[iso2_code] = pais.alpha_3
    except KeyError:
        print(f"Aviso: Não foi possível encontrar o código ISO-3 para '{iso2_code}'.")
dfmedia['iso_alpha3'] = dfmedia['residencia'].map(iso3_dict)

print("Dicionário de conversão ISO-2 para ISO-3:")
dfmedia

Dicionário de conversão ISO-2 para ISO-3:


,residencia,usd,iso_alpha3
0,AD,50745.000000,AND
1,AE,88333.333333,ARE
2,AM,45453.200000,ARM
3,AR,78856.641026,ARG
4,AS,45555.000000,ASM
...,...,...,...
97,VE,192500.000000,VEN
98,VN,57200.000000,VNM
99,XK,41052.000000,NaN
100,ZA,65204.641791,ZAF


In [61]:
df = px.data.gapminder()
fig = px.choropleth(
    dfmedia
    , locations="iso_alpha3"
    , color="usd"
    , scope="world"
    , title= "Média Salarial de Cientista de Dados por País"
)
fig.show()